In [ ]:
from ultralytics import YOLO

# Cargar modelo preentrenado en COCO (80 clases)
model = YOLO("yolov8n.pt")

# Hacer inferencia
results = model("imagen.jpg")

# Mostrar predicciones
results.show()
